Purpose: This gets the APN file from Kern County and joins it with the TWSC layer from AEWSD and filters it to the district boundaries


In [1]:
from data import district_boundary
import geopandas as gpd
from pathlib import Path

data_path = Path("../data")
apns = gpd.read_parquet(data_path.joinpath("apns.parquet")).to_crs("EPSG:4326")
apn_gdf = apns[apns.intersects(district_boundary.unary_union)]
apn_gdf.head(3)

ModuleNotFoundError: No module named 'data'

In [2]:
TW = gpd.read_file(data_path.joinpath("TWSC_2023/TWSC_2023.shp"))
GW = gpd.read_file(data_path.joinpath("FrickUnit_Outreach/GWSA.shp"))

TW["APN"].unique()

GW["APN"].unique()

In [11]:
apn_gdf["Service Area Type"] = (
    apn_gdf["APN"]
    .str.replace("-", "")
    .apply(
        lambda x: "Groundwater and Temporary Water"
        if x in TW["APN"].unique() and x in GW["APN"].unique()
        else "Temporary Water"
        if x in TW["APN"].unique()
        else "Groundwater"
        if x in GW["APN"].unique()
        else "Surface Water"
    )
)

c:\Users\Denver\AppData\Local\pypoetry\Cache\virtualenvs\pandas-analysis-pSGX_PKJ-py3.10\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
apn_gdf["Service Area Type"].value_counts()

Groundwater                        8398
Surface Water                      3230
Groundwater and Temporary Water     320
Temporary Water                      34
Name: Service Area Type, dtype: int64

In [13]:
apn_gdf.to_parquet(data_path.joinpath("apn_gdf_sa.parquet"))